In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm

from statsmodels.regression.linear_model import OLS

In [2]:
df = pd.read_csv("Data_USDEUR.csv")

In [3]:
max = len(df)
for index, row in df.iterrows():
    if index < max - 1 and abs(df.loc[index, 'Time'] - df.loc[index + 1, 'Time']) < 1e-6:
        # print(index)
        df.loc[index + 1, 'Volume'] = df.loc[index + 1, 'Volume'] + df.loc[index, 'Volume']
        df.loc[index + 1, 'AskBefore'] = df.loc[index, 'AskBefore']
        df = df.drop([index])



In [4]:
df = df.reset_index()

In [5]:
df = df.drop(["index"], axis=1)

In [6]:
df.to_csv("DataWithoutPairs_USDEUR.csv")

In [65]:
df = pd.read_csv("DataWithoutPairs_USDRUB.csv")

Функция оценивает регрессию и вычисляет по ней параметры.

In [66]:
def df_OLS_anal(df: pd.DataFrame):
    # max = len(df)
    # for index, row in df.iterrows():
    #     if index < max - 1 and abs(df.loc[index, 'Time'] - df.loc[index + 1, 'Time']) < 1e-6:
    #         # print(index)
    #         df.loc[index + 1, 'Volume'] = df.loc[index + 1, 'Volume'] + df.loc[index, 'Volume']
    #         df.loc[index + 1, 'AskBefore'] = df.loc[index, 'AskBefore']
    #         df = df.drop([index])

    deltaA = df["AskAfter"][1:] - df["AskBefore"][1:]
    deltat = np.diff(df["Time"])
    x = df["Volume"][1:]

    dA = deltaA / deltat
    dx = x / deltat

    ddx = np.diff(dx)
    ddA = np.diff(dA)

    OLSdf = pd.DataFrame({
        "SUM": ddA,
        "-rho": deltaA[:-1],
        "rho lambda": x[:-1],
        "kappa + lambda": ddx
    })

    Regressand = OLSdf["SUM"]

    Regressors = OLSdf[["-rho", "rho lambda", "kappa + lambda"]]

    Regressors = sm.add_constant(Regressors, prepend=False)

    model = OLS(Regressand, Regressors)

    res = model.fit()

    print(res.summary()) 

    rho = - res.params.iloc[0]
    lamb = res.params.iloc[1] / rho
    kappa = res.params.iloc[2] - lamb

    print("rho = ", rho, "\nlambda = ", lamb, "\nkappa = ", kappa, "\n \n", end="")

In [67]:
df_OLS_anal(df)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     144.3
Date:                Thu, 18 Jan 2024   Prob (F-statistic):           1.74e-91
Time:                        16:46:22   Log-Likelihood:                -90059.
No. Observations:                9743   AIC:                         1.801e+05
Df Residuals:                    9739   BIC:                         1.802e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho           -1.503e+04    981.693    -15.

In [68]:
df = pd.read_csv("DataWithoutPairs.csv")
df = df[df["Time"] < 3600]
df = df[df["Time"] > 40]
df = df.reset_index()
df_OLS_anal(df)
df.head(3)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     50.83
Date:                Thu, 18 Jan 2024   Prob (F-statistic):           1.35e-31
Time:                        16:46:45   Log-Likelihood:                -16746.
No. Observations:                1905   AIC:                         3.350e+04
Df Residuals:                    1901   BIC:                         3.352e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho           -4505.8608   1273.953     -3.

,index,Unnamed: 0.1,Unnamed: 0,Time,AskBefore,AskAfter,Volume
0,224,224,274,41.000686,61.857,61.857,1000.0
1,225,225,275,44.000950,61.850,61.857,1000.0
2,226,226,277,45.000454,61.857,61.860,23000.0


In [69]:
df = pd.read_csv("DataWithoutPairs.csv")
df.Time *= 1000
df_OLS_anal(df)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     144.3
Date:                Thu, 18 Jan 2024   Prob (F-statistic):           1.74e-91
Time:                        16:46:52   Log-Likelihood:                -22757.
No. Observations:                9743   AIC:                         4.552e+04
Df Residuals:                    9739   BIC:                         4.555e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho             -15.0336      0.982    -15.

In [70]:
df = pd.read_csv("DataWithoutPairs.csv")
df = df[df["AskAfter"] - df["AskBefore"] > 0.01]
df = df.reset_index()
df = df.drop(["index"], axis=1)
df_OLS_anal(df)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     49.29
Date:                Thu, 18 Jan 2024   Prob (F-statistic):           2.24e-28
Time:                        16:47:06   Log-Likelihood:                -5791.6
No. Observations:                 582   AIC:                         1.159e+04
Df Residuals:                     578   BIC:                         1.161e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho           -3158.8036   2224.086     -1.

In [71]:
df = pd.read_csv("DataWithoutPairs.csv")
df = df[df["Volume"] > 100000]
df = df.reset_index()
df_OLS_anal(df)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     197.5
Date:                Thu, 18 Jan 2024   Prob (F-statistic):          1.67e-106
Time:                        16:47:30   Log-Likelihood:                -11301.
No. Observations:                1380   AIC:                         2.261e+04
Df Residuals:                    1376   BIC:                         2.263e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho           -2.513e+04   4013.482     -6.

In [72]:
df = pd.read_csv("DataWithoutPairs.csv")
df = df[df["Volume"] > 10000]
df = df.reset_index()
df = df[df["AskAfter"] - df["AskBefore"] < 0.1]
df = df.reset_index()
df_OLS_anal(df)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     113.1
Date:                Thu, 18 Jan 2024   Prob (F-statistic):           3.04e-70
Time:                        16:47:41   Log-Likelihood:                -37846.
No. Observations:                3955   AIC:                         7.570e+04
Df Residuals:                    3951   BIC:                         7.572e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho           -8.673e+04   5609.339    -15.